In [268]:
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.preprocessing import StandardScaler

import warnings

# Your code generating the warning
# For example:
# warnings.warn("This is a warning")

# To silence all warnings temporarily:
warnings.filterwarnings("ignore", message="A value is trying to be set on a copy of a slice from a DataFrame.")

    
    # Your code generating the warning goes here

# To silence a specific type of warning:
# warnings.filterwarnings("ignore", category=YourWarningCategory)


In [253]:
df_train=pd.read_csv("train_combined_with_biobert.csv")
df_test=pd.read_csv("test_combined_with_biobert.csv")

In [254]:
drop_col=["index","index.1","delivery_mode_NVD"]

y_train=df_train['delivery_mode_NVD']
y_temp=df_test['delivery_mode_NVD']

X_train=df_train.drop(columns=drop_col)
X_temp=df_test.drop(columns=drop_col)

In [255]:
# !pip install tensorflow

In [256]:
# Split the temporary set into validation and test set (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


#### Simple FNN

In [257]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Activation,Dropout

# input_shape=X_train.shape[1]
# # Define the model architecture
# model = Sequential([
#     Dense(128, input_shape=(input_shape,), activation='relu'),
#     Dense(64,  activation='relu'),  # Input layer with 64 neurons and ReLU activation
#     Dense(32, activation='relu'),
#     # Dropout(0.2),# Hidden layer with 32 neurons and ReLU activation
#     Dense(1, activation='sigmoid')  # Output layer with 1 neuron and sigmoid activation for binary classification
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC','Accuracy','Precision','Recall'])

# # Train the model
# model.fit(X_train, y_train, epochs=30, batch_size=32,validation_data=(X_val, y_val))

# # # Evaluate the model
# model.evaluate(X_test, y_test)


#### Code to implement cross validation and standard scaling on FNN

In [258]:
X=pd.concat([X_train,X_test],axis=0)
y=pd.concat([y_train,y_test],axis=0)

X=X.reset_index()
y=y.reset_index()
X=X.drop(columns=["index",'Unnamed: 0'])
y=y.drop(columns=["index"])

In [263]:
##Numerical columns:

int_cols=['12', '13', '14', '20', '21', '22', '45', '46', '47', '30', '34', '32', '33']

In [264]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.469949,-0.342463,1.475144,-0.894011,-0.304161,-0.635746,-0.040267,-0.290035,0.129460,-0.096047,-0.211023,0.150510,21.0,12.1,37.0,0.0,0.0,0.0,0.0,0.0,56.0,0.56,178.571429,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.60,8.0,2671.00000,0.0,0.0
1,0.620910,-0.229851,-0.527119,-0.697184,1.426482,-0.376539,-0.623211,-0.214972,-0.126768,0.575531,-0.752973,0.055201,27.0,9.6,32.0,0.0,0.0,0.0,0.0,0.0,40.0,0.40,250.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.40,3.5,1487.00000,0.0,0.0
2,0.882696,-0.130102,-1.696773,0.567934,-0.927484,0.589872,-0.082100,0.689685,0.075018,-0.363091,-0.171727,-0.266925,29.0,11.1,36.0,0.0,0.0,0.0,0.0,0.0,85.0,0.85,117.647059,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.96,7.3,2722.00000,0.0,0.0
3,-1.008237,-0.728161,0.652063,0.220852,-0.044954,-0.875489,0.373319,0.328383,-0.126669,-0.423650,-0.661616,0.437395,27.0,11.1,37.0,0.0,0.0,0.0,0.0,0.0,55.0,0.55,181.818182,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.30,16.7,2728.00000,0.0,0.0
4,0.888258,-1.592506,-0.233333,0.311682,-0.617630,1.388027,-0.042201,-0.041561,-0.735091,-0.379480,-0.263331,0.295628,21.0,10.4,35.0,0.0,0.0,0.0,0.0,0.0,58.0,0.58,172.413793,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.20,4.2,1912.63631,0.0,0.0


In [269]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

# Define your FNN architecture
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Define the number of folds for cross-validation
n_folds = 10

# Initialize the cross-validation splitter
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Perform cross-validation
auc_scores = []
for train_index, val_index in skf.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    
    scaler = StandardScaler()

    # Fit the scaler on the numerical columns of the training data and transform
    X_train[int_cols] = scaler.fit_transform(X_train[int_cols])

    # Transform the numerical columns of the test data using the same scaler
    X_val[int_cols] = scaler.transform(X_val[int_cols])
    
    # Build and train the model
    model = build_model()
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    
    # Predict probabilities on validation set
    y_pred_proba = model.predict(X_val)
    
    # Calculate AUC score
    auc_score = roc_auc_score(y_val, y_pred_proba)
    auc_scores.append(auc_score)

# Calculate mean AUC score
mean_auc_score = np.mean(auc_scores)
print(f'Mean AUC Score: {mean_auc_score:.4f}')


/tmp/ipykernel_8156/1844098197.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[int_cols] = scaler.fit_transform(X_train[int_cols])
/tmp/ipykernel_8156/1844098197.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[int_cols] = scaler.transform(X_val[int_cols])


1/1 [==============================] - 0s 50ms/step


/tmp/ipykernel_8156/1844098197.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[int_cols] = scaler.fit_transform(X_train[int_cols])
/tmp/ipykernel_8156/1844098197.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[int_cols] = scaler.transform(X_val[int_cols])


1/1 [==============================] - 0s 54ms/step


/tmp/ipykernel_8156/1844098197.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[int_cols] = scaler.fit_transform(X_train[int_cols])
/tmp/ipykernel_8156/1844098197.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[int_cols] = scaler.transform(X_val[int_cols])


...

1/1 [==============================] - 0s 100ms/step


/tmp/ipykernel_8156/1844098197.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[int_cols] = scaler.fit_transform(X_train[int_cols])
/tmp/ipykernel_8156/1844098197.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[int_cols] = scaler.transform(X_val[int_cols])


1/1 [==============================] - 0s 58ms/step


/tmp/ipykernel_8156/1844098197.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[int_cols] = scaler.fit_transform(X_train[int_cols])
/tmp/ipykernel_8156/1844098197.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[int_cols] = scaler.transform(X_val[int_cols])


1/1 [==============================] - 0s 54ms/step
Mean AUC Score: 0.8612


In [ ]:
## Drop out and regularizarion added

In [235]:
print(skf)

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)


In [233]:
auc_scores

[0.7705426356589147,
 0.8651162790697674,
 0.8205980066445183,
 0.8365079365079364,
 0.9015873015873016]